# IRSA AllWISE Example (SIA2)

In [225]:
import requests #http://docs.python-requests.org/en/master/
from io import BytesIO
from astropy.table import Table
import firefly_client #https://firefly-client.lsst.io/v/DM-10984-user-guide/index.html
import shutil

## Send a URL request to IRSA.

In [226]:
r = requests.get("https://irsa.ipac.caltech.edu/SIA?COLLECTION=wise_allwise" + 
                       "&POS=circle+-164.7+-5.8+0.002777&RESPONSEFORMAT=fits")

## Put the results in an astropy table. 
http://docs.astropy.org/en/stable/table/access_table.html

In [227]:
table = Table.read(BytesIO(r.content), hdu=1)

## Give me some basic info about the table that was returned.

In [228]:
table.info

/Users/desai/anaconda/lib/python3.5/site-packages/astropy/table/column.py:268: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return self.data.__eq__(other)


<Table length=27>
           name            dtype  shape  unit n_bad
------------------------- ------- ----- ----- -----
      null_bitfield_flags   uint8  (6,)           0
                 s_region  str181         deg     0
           obs_collection   str12                 0
               obs_intent    str7                 0
           algorithm_name    str6                 0
           telescope_name    str4                 0
       telescope_keywords    str1                 0
          instrument_name    str4                 0
      instrument_keywords    str1                 0
  environment_photometric    bool                 0
              proposal_id    str1                 0
              proposal_pi   str16                 0
         proposal_project    str4                 0
           proposal_title   str58                 0
              target_name    str1                 0
              target_type    str5                 0
          target_standard    bool             

# Print a selection of columns.

In [229]:
print(table['energy_bandpassname', 'access_format'])

energy_bandpassname   access_format   
------------------- ------------------
                 W3 application/x-gzip
                 W3 image/fits        
                 W3 text/plain        
                 W3 text/plain        
                 W3 application/x-gzip
                 W3 text/plain        
                 W3 text/plain        
                 W1 text/plain        
                 W1 text/plain        
                 W1 image/fits        
                ...                ...
                 W2 text/plain        
                 W2 application/x-gzip
                 W2 application/x-gzip
                 W2 image/fits        
                 W2 text/plain        
                 W2 text/plain        
                 W4 text/plain        
                 W4 application/x-gzip
                 W4 image/fits        
                 W4 application/x-gzip
                 W4 text/plain        
Length = 27 rows


## Choose FITS files taken in the W1 filter.

In [230]:
index = (table['energy_bandpassname'] == 'W1') & (table['access_format'] == 'image/fits        ')

In [231]:
table[index]

null_bitfield_flags [6],s_region,obs_collection,obs_intent,algorithm_name,telescope_name,telescope_keywords,instrument_name,instrument_keywords,environment_photometric,proposal_id,proposal_pi,proposal_project,proposal_title,target_name,target_type,target_standard,target_moving,target_keywords,calib_level,dataproduct_type,position_dimension_naxis1,position_dimension_naxis2,position_samplesize,position_timedependent,energy_emband,energy_bandpassname,t_min,t_max,t_exptime,t_resolution,access_url,access_format,access_estsize,obs_id,obs_publisher_did,s_ra,s_dec,s_fov,s_resolution,em_min,em_max,em_res_power,o_ucd
,deg,,,,,,,,,,,,,,,,,,,,,,deg,,,,d,d,s,s,,,kbyte,,,deg,deg,deg,deg,m,m,,
uint8,str181,str12,str7,str6,str4,str1,str4,str1,bool,str1,str16,str4,str58,str1,str5,bool,bool,str1,int16,str5,int64,int64,float64,bool,str8,str2,float64,float64,float64,float64,str113,str18,int64,str36,str42,float64,float64,float64,float64,float64,float64,float64,str1
5 .. 64,POLYGON ICRS -163.261860888792 -6.839647613318 -164.83756839366 -6.839648167653 -164.835289092986 -5.275156783985 -163.264141302672 -5.275156231255 -163.261860888792 -6.839647613318,wise_allwise,science,mosaic,WISE,,WISE,,True,,Edward L. Wright,WISE,Wide-field Infrared Survey Explorer - AllWISE Atlas Images,,field,False,False,,3,image,4095,4095,2.75,False,Infrared,W1,55203.0879785,55570.6052799,196.3962,nan,https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/19/1959/1959m061_ac51/1959m061_ac51-w1-int-3.fits,image/fits,9223372036854775807,3a1d748d-67ff-423a-9868-7971693dfc03,ivo://irsa.ipac/wise_allwise/1959m061_ac51,-164.049907,-6.057778,3.128125,6.1,3.15e-06,3.78e-06,nan,


## Get URL of file that you want.

In [252]:
url = table['access_url'][index][0].strip()

In [253]:
url

'https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/19/1959/1959m061_ac51/1959m061_ac51-w1-int-3.fits'

In [254]:
response = requests.get(url, stream=True)
with open('image.fits', 'wb') as out_file:
    shutil.copyfileobj(response.raw,out_file)
    